# Purpose

I used this notebook to work out a stratgey to blend the 4 red, green, blue and yellow images into a single image with Pillow.

In [ ]:


import numpy as np 
import pandas as pd 

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from PIL import Image  
from IPython.display import display 

import os
import datetime

## Parameters

In [ ]:
ROOT_PATH = '/kaggle/input/hpa-single-cell-image-classification/'

CHANNELS = np.array(['red', 'green', 'blue', 'yellow'])

CHANNEL_SIZE = len(CHANNELS)
SAMPLE_SIZE = 40

# Get Images
Return array of images for all channels

In [ ]:
def get_images(id):
    images = list()
    for channel in CHANNELS:
        path = ROOT_PATH + 'train/{}_{}.png'.format(id, channel)
        image = Image.open(path) 
        images.append(image)
    return images

## Read Training Data

In [ ]:
df_train = pd.read_csv(ROOT_PATH + 'train.csv')
print("Trainning data length: {}".format(len(df_train)))
df_train.head()

In [ ]:
# if sample size is set then reduce trainning set accordingly
if SAMPLE_SIZE > -1:
    df_train = df_train.sample(SAMPLE_SIZE)
    df_train.reset_index(inplace=True);

# Exploration

Some exploration to get to a final solution to blend images

In [ ]:
images = get_images( df_train.loc[df_train.index[0], 'ID'] )

# zeros array to fill in missing RGB channels
size = np.shape(images[0])[0]
zeros = np.zeros((size, size))

# Create arrays for each image with 3 channels 
red_array = np.stack((images[0], zeros, zeros), 2)
green_array = np.stack((zeros, images[1], zeros), 2)
blue_array = np.stack((zeros, zeros, images[2]), 2)
# Place yellow across both red and green channels
yellow_array = np.stack((images[3], images[3], zeros), 2)

# Convert to image
red_image = Image.fromarray( np.uint8(red_array) )
green_image = Image.fromarray( np.uint8(green_array) )
blue_image = Image.fromarray( np.uint8(blue_array) )
yellow_image = Image.fromarray( np.uint8(yellow_array) )

# Plot what we have so far
fig = plt.figure(figsize=(25,25))
ax = fig.add_subplot(1, CHANNEL_SIZE, 1)
ax.set_title("Red")
plt.imshow(np.asarray(red_image))

ax = fig.add_subplot(1, CHANNEL_SIZE, 2)
ax.set_title("Green")
plt.imshow(np.asarray(green_image))

ax = fig.add_subplot(1, CHANNEL_SIZE, 3)
ax.set_title("Blue")
plt.imshow(np.asarray(blue_image))

ax = fig.add_subplot(1, CHANNEL_SIZE, 4)
ax.set_title("Yellow")
plt.imshow(np.asarray(yellow_image))

# Copy over yellow into red and blue taking the max value of each pixel between the two
# Red / Yellow
red_yellow_array = np.maximum(red_array, yellow_array)
red_yellow_image = Image.fromarray( np.uint8(red_yellow_array) )
ax = fig.add_subplot(2, CHANNEL_SIZE, 5)
ax.set_title("Red / Yellow")
plt.imshow(np.asarray(red_yellow_image))

# Green / Yellow
green_yellow_array = np.maximum(green_array, yellow_array)
green_yellow_image = Image.fromarray( np.uint8(green_yellow_array) )
ax = fig.add_subplot(2, CHANNEL_SIZE, 6)
ax.set_title("Green / Yellow")
plt.imshow(np.asarray(green_yellow_image))

# Final solution to blend RGBY into RGB
blended_array = np.stack((
        np.maximum(images[0], images[3]),
        np.maximum(images[1], images[3]),
        images[2]
    ), 2)
blended_image = Image.fromarray( np.uint8(blended_array) )
ax = fig.add_subplot(2, CHANNEL_SIZE, 7)
ax.set_title("Final")
plt.imshow(np.asarray(blended_image))


# Final Solution
Plot blended images for sampled images

In [ ]:
def plot_blended_image(df_samples):
    
    fig = plt.figure(figsize=(25,2*len(df_samples)))
    index = 1
    
    for sample_index, sample in df_samples.iterrows():

        # get rgby images for sample
        images = get_images(sample['ID'])
        
        # blend rgby images into single array
        blended_array = np.stack((
                np.maximum(images[0], images[3]),
                np.maximum(images[1], images[3]),
                images[2]
            ), 2)
        blended_image = Image.fromarray( np.uint8(blended_array) )
        
        # plot
        ax = fig.add_subplot(len(df_samples) // 4 + 1, 4, index)
        plt.imshow(np.asarray(blended_image))

        index = index + 1

In [ ]:
plot_blended_image(df_train)